# Koneksi Google Drive and Folders


In [1]:
from google.colab import drive
drive.mount('/content/drive')

data_root='/content/drive/MyDrive/Chatbot/'



Mounted at /content/drive


# Import Library

In [2]:
import json
# untuk membuat dan membaca (konsumsi) data JSON.

import string
# Modul Python String berisi beberapa konstanta, fungsi utilitas, dan kelas untuk manipulasi string.

import random
# Modul random pada python berfungsi untuk menghasilkan angka acak yang bisa kamu gunakan untuk berbagai keperluan

import nltk
 #  singkatan dari Natural Language Tool Kit, yaitu sebuah library yang digunakan untuk membantu kita dalam bekerja dengan teks.
#  Library ini memudahkan kita untuk memproses teks seperti melakukan classification, tokenization, stemming, tagging, parsing, dan semantic reasoning.

import numpy as np
# mendukung array (termasuk array multidimensi) serta menyediakan koleksi fungsi matematika untuk mengoperasikan array tersebut

from nltk.stem import WordNetLemmatizer
# Salah satu modulnya adalah WordNet Lemmatizer yang dapat digunakan untuk melakukan lemmatisasi pada kata-kata.
#Lemmatisasi adalah proses mengelompokkan berbagai bentuk infleksi suatu kata sehingga dapat dianalisis sebagai satu item. Lemmatisasi mirip dengan stemming tetapi
#membawa konteks pada kata-katanya. Jadi itu menghubungkan kata-kata dengan arti yang mirip dengan satu kata.

import tensorflow as tf
# Mempersiapkan data latih dan target. Mengonfigurasi parameter pelatihan seperti jumlah epoch, batch size, dan lainnya.

from tensorflow.keras import Sequential
# digunakan untuk mengimpor kelas Sequential dari modul keras dalam pustaka tensorflow,
#Sequential adalah bagian dari API Keras, yang merupakan antarmuka tingkat tinggi untuk membangun dan melatih model neural network.


from tensorflow.keras.layers import Dense, Dropout
# Dropout adalah teknik regulasi yang umum digunakan untuk mencegah overfitting dalam model neural network.
# from tensorflow.keras.layers import Dense, Dropout digunakan untuk mengimpor dua jenis lapisan (layers)
# yang umum digunakan dalam pembangunan model neural network menggunakan pustaka tensorflow.keras



nltk.download("punkt")
# digunakan untuk mengunduh data Punkt, yang merupakan model tokenisasi yang disediakan oleh NLTK (Natural Language Toolkit) di Python.
# Tokenisasi adalah proses membagi teks menjadi token atau unit-unit kecil seperti kata atau kalimat. Dengan mengunduh data Punkt
# download database corpus dari nltk. Proses ini hanya sekali ketika running awal.


nltk.download("wordnet")

#  digunakan untuk mengunduh data WordNet melalui NLTK (Natural Language Toolkit) di Python. WordNet adalah sebuah kamus bahasa Inggris yang
#  memiliki struktur hierarki dan mengelompokkan kata-kata berdasarkan makna dan hubungan antar kata tersebut.


nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Reading the JSON file

In [3]:
data_file = open(data_root + '/intents.json').read()

data = json.loads(data_file)

# Identifikasi Fitur & target model

Penggunaan NLTK: Kode menggunakan modul NLTK untuk tokenisasi kata dan WordNetLemmatizer untuk melakukan lemmatisasi kata-kata. Tokenisasi membantu dalam memecah pola-pola menjadi kata-kata individual, sedangkan lemmatisasi mengonversi kata-kata ke bentuk dasarnya.


Menghilangkan tanda baca dan mengurutkan kata-kata: Kode juga menghilangkan tanda baca dan mengurutkan kata-kata yang telah dilematisasi untuk memastikan bahwa setiap kata hanya muncul sekali.


1. case folding = penyeragaman karakter (lower)
2. filtering = karakter yang dianggap ilegal
3. tokenization = jika tidak melakukan ini maka tidak bisa mengecek setiap kata (tidak bisa detect kata penting / baku) masing2 kata berdiri memisahkan kalimat menjadi kata perkata berdiri sendiri
4. slangword convertion = menjadikan kata dasar misal cepat cepet
5. stopword removal =menghapus kata imbuhan
6. stemming = mengubah imbuhan ke bentuk dasar (imbuhan bahasa indonesia)
7. lemmatization = mengubah imbuhan ke bentuk dasar bentuknya kamus (imbuhan bahasa indonesia + inggris kata baku)
8. padding = data teks harus sama (deep learning)

# Feature Text

In [4]:

# getting informations from intents.json--
words = []     # Daftar kata-kata dari pola-pola
classes = []   # Daftar kelas
data_x = []    # Pola-pola
data_y = []    # Kelas yang sesuai dengan pola-pola

for intent in data['intents']:
    for pattern in intent['patterns']:

        # Tokenisasi kata dari setiap pola
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"]),

        if intent["tag"] not in classes:
          classes.append(intent["tag"])


lemmatizer = WordNetLemmatizer()
# Membuat objek lemmatizer dari WordNetLemmatizer

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# Melakukan lematisasi pada setiap kata dalam daftar 'words' dan mengonversi ke huruf kecil

words = sorted(set(words))
# Mengurutkan dan menghapus duplikat kata-kata dalam daftar 'words'

classes = sorted(set(classes))
# Mengurutkan dan menghapus duplikat niat (classes) dalam daftar 'classes'

# Preprocessing Data

In [5]:
training = []
# Membuat list kosong untuk menyimpan data pelatihan

out_empty = [0] * len(classes)
# Membuat list kosong berisi nol sebanyak jumlah kelas


for idx, doc in enumerate(data_x):
# Iterasi melalui setiap dokumen (pola) dalam data_x

  bow = []
# Inisialisasi BoW (Bag of Words) untuk dokumen saat ini

  text = lemmatizer.lemmatize(doc.lower())
# Melakukan lematisasi dan mengonversi teks ke huruf kecil

  for word in words:
# Iterasi melalui setiap kata dalam daftar kata-kata yang telah diproses sebelumnya

    bow.append(1) if word in text else bow.append(0)
# Jika kata dalam BoW, tambahkan 1, jika tidak, tambahkan 0

    output_row = list(out_empty)
# Inisialisasi baris output dengan list nol sesuai jumlah kelas

    output_row[classes.index(data_y[idx])] = 1
# Mengatur elemen sesuai dengan indeks kelas yang sesuai untuk output_row

    training.append([bow, output_row])
# Menambahkan pasangan BoW dan output_row ke dalam data pelatihan


random.shuffle(training)
# Mengacak urutan data pelatihan

training = np.array(training, dtype=object)
# Mengonversi list training ke dalam array NumPy dengan tipe data objek


train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Mengonversi array NumPy ke dalam train_x dan train_y

Sequential adalah model neural network yang umum digunakan dalam Keras, yang merupakan bagian dari TensorFlow. Model ini digunakan untuk membangun jaringan neural secara sekuensial, satu lapisan setelah yang lain.

Pada model Sequential, setiap layer terhubung ke layer sebelumnya, dan informasi mengalir secara sekuensial melalui model tersebut.

# Modelling

In [6]:
model = Sequential()
# Membuat model Sequential

model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
# Menambahkan layer Dense pertama dengan 128 neuron, input_shape sesuai dengan panjang train_x[0], dan fungsi aktivasi ReLU

model.add(Dropout(0.5))
# Menambahkan layer Dropout dengan tingkat dropout 0.5

model.add(Dense(64, activation="relu"))
# Menambahkan layer Dense kedua dengan 64 neuron dan fungsi aktivasi ReLU

model.add(Dropout(0.5))
# Menambahkan layer Dropout dengan tingkat dropout 0.5

model.add(Dense(len(train_y[0]), activation = "softmax"))
# Menambahkan layer Dense keluaran dengan jumlah neuron sesuai dengan panjang train_y[0] dan fungsi aktivasi softmax

adam = tf.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
# Menggunakan optimizer Adam dengan learning rate 0.01 dan decay 1e-6

# Mengompilasi model dengan fungsi loss categorical_crossentropy, optimizer Adam, dan metrik akurasi
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
# Menampilkan ringkasan model

model.fit(x=train_x, y=train_y, epochs=30, verbose=1)
# Melatih model dengan data pelatihan (train_x dan train_y) selama 30 epochs


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               38656     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 106)               6890      
                                                                 
Total params: 53802 (210.16 KB)
Trainable params: 53802 (210.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/30
39

# Building the Neural Network Model

In [7]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

# Fungsi untuk membersihkan teks dengan tokenisasi dan lematisasi


def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
    return np.array(bow)

# Fungsi untuk representasi Bag of Words dari teks berdasarkan vokabuler tertentu

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow])) [0]
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list
# Fungsi untuk memprediksi kelas (intent) dari teks menggunakan model


def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Maaf! saya tidak mengerti."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

# Fungsi untuk memprediksi kelas (intent) dari teks menggunakan model


# Pre-processing the User’s Input

In [9]:
# menampilkan hasil
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
  message = input("")
  if message == "0":
    break
  intents = pred_class(message, words, classes)
  result = get_response(intents, data)
  0
  print(result)

Press 0 if you don't want to chat with our ChatBot.
0


In [10]:
model.save('chatbot.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
tokenizer = Tokenizer(num_word=2000)
tokenizer.fit_on_texts(data['pattern'])
train = tokenizer.text_to_sequence(data['pattern'])
train